<a href="https://colab.research.google.com/github/mocamocamo/inflearn-llm-colab/blob/main/4.%20Char_RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_Language_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 라이브러리 import


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

# 유틸리티 함수 정의

In [ ]:
# input 데이터와 input 데이터를 한글자씩 뒤로 민 target 데이터를 생성하는 utility 함수를 정의합니다.
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]

  return input_text, target_text

# 설정값 지정

In [ ]:
# 학습에 필요한 설정값들을 지정합니다.
#data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')  # shakespeare
data_dir = tf.keras.utils.get_file('linux.txt', 'https://raw.githubusercontent.com/solaris33/deep-learning-tensorflow-book-code/master/Ch08-RNN/Char-RNN/data/linux/input.txt')  # linux
batch_size = 64      # Training : 64, Sampling : 1
seq_length = 100     # Training : 100, Sampling : 1
embedding_dim = 256  # Embedding 차원
hidden_size = 1024   # 히든 레이어의 노드 개수
num_epochs = 10

6209536/6206996 [==============================] - 0s 0us/step


# 어휘 집합(Vocabulary set) 설정

In [ ]:
# 학습에 사용할 txt 파일을 읽습니다.
text = open(data_dir, 'rb').read().decode(encoding='utf-8')
# 학습데이터에 포함된 모든 character들을 나타내는 변수인 vocab과
# vocab에 id를 부여해 dict 형태로 만든 char2idx를 선언합니다.
vocab = sorted(set(text))  # 유니크한 character 개수
vocab_size = len(vocab)
print('{} unique characters'.format(vocab_size))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

99 unique characters


# Dataset 설정

In [ ]:
# 학습 데이터를 character에서 integer로 변환합니다.
text_as_int = np.array([char2idx[c] for c in text])

# split_input_target 함수를 이용해서 input 데이터와 input 데이터를 한글자씩 뒤로 민 target 데이터를 생성합니다.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

# RNN 모델 설정

In [ ]:
# tf.keras.Model을 이용해서 RNN 모델을 정의합니다.
class RNN(tf.keras.Model):
 def __init__(self, batch_size):
   super(RNN, self).__init__()
   self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                    batch_input_shape=[batch_size, None])
   self.hidden_layer_1 = tf.keras.layers.LSTM(hidden_size,
                                             return_sequences=True,
                                             stateful=True,
                                             recurrent_initializer='glorot_uniform')
   self.output_layer = tf.keras.layers.Dense(vocab_size)

 def call(self, x):
   embedded_input = self.embedding_layer(x)
   features = self.hidden_layer_1(embedded_input)
   logits = self.output_layer(features)

   return logits

# Loss Function 정의

In [ ]:
# sparse cross-entropy 손실 함수를 정의합니다.
def sparse_cross_entropy_loss(labels, logits):
  return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True))

# 옵티마이저 및 학습 설정

In [ ]:
# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.keras.optimizers.Adam()

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, input, target):
  with tf.GradientTape() as tape:
    logits = model(input)
    loss = sparse_cross_entropy_loss(target, logits)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

# 샘플링 함수 설정

In [ ]:
def generate_text(model, start_string):
  num_sampling = 4000  # 생성할 글자(Character)의 개수를 지정합니다.

  # start_sting을 integer 형태로 변환합니다.
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 샘플링 결과로 생성된 string을 저장할 배열을 초기화합니다.
  text_generated = []

  # 낮은 temperature 값은 더욱 정확한 텍스트를 생성합니다.
  # 높은 temperature 값은 더욱 다양한 텍스트를 생성합니다.
  temperature = 1.0

  # 여기서 batch size = 1 입니다.
  model.reset_states()
  for i in range(num_sampling):
    predictions = model(input_eval)
    # 불필요한 batch dimension을 삭제합니다.
    predictions = tf.squeeze(predictions, 0)

    # 모델의 예측결과에 기반해서 랜덤 샘플링을 하기위해 categorical distribution을 사용합니다.
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # 예측된 character를 다음 input으로 사용합니다.
    input_eval = tf.expand_dims([predicted_id], 0)
    # 샘플링 결과를 text_generated 배열에 추가합니다.
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

# 트레이닝 시작

In [ ]:
# Recurrent Neural Networks(RNN) 모델을 선언합니다.
RNN_model = RNN(batch_size=batch_size)

# 데이터 구조 파악을 위해서 예제로 임의의 하나의 배치 데이터 에측하고, 예측결과를 출력합니다.
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = RNN_model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# 모델 정보를 출력합니다.
RNN_model.summary()

# checkpoint 데이터를 저장할 경로를 지정합니다.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

for epoch in range(num_epochs):
  start = time.time()

  # 매 반복마다 hidden state를 초기화합니다. (최초의 hidden 값은 None입니다.)
  hidden = RNN_model.reset_states()

  for (batch_n, (input, target)) in enumerate(dataset):
    loss = train_step(RNN_model, input, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
  if (epoch + 1) % 5 == 0:
    RNN_model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

RNN_model.save_weights(checkpoint_prefix.format(epoch=epoch))
print("트레이닝이 끝났습니다!")

(64, 100, 99) # (batch_size, sequence_length, vocab_size)
Model: "rnn_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  25344     
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
dense_2 (Dense)              multiple                  101475    
Total params: 5,373,795
Trainable params: 5,373,795
Non-trainable params: 0
_________________________________________________________________
Epoch 1 Batch 0 Loss 4.595424652099609
Epoch 1 Batch 100 Loss 2.8029003143310547
Epoch 1 Batch 200 Loss 2.368008613586426
Epoch 1 Batch 300 Loss 1.9701776504516602
Epoch 1 Batch 400 Loss 1.740452527999878
Epoch 1 Batch 500 Loss 1.6691888570785522
Epoch 1 Batch 600 Loss 1.5763310194015503
Epoch 1 Batch 700 Loss 1.513190031

# 트레이닝이 끝난 모델을 이용한 샘플링

In [ ]:
sampling_RNN_model = RNN(batch_size=1)
sampling_RNN_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
sampling_RNN_model.build(tf.TensorShape([1, None]))
sampling_RNN_model.summary()

# 샘플링을 시작합니다.
print("샘플링을 시작합니다!")
print(generate_text(sampling_RNN_model, start_string=u' '))

Model: "rnn_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  25344     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  5246976   
_________________________________________________________________
dense_3 (Dense)              multiple                  101475    
Total params: 5,373,795
Trainable params: 5,373,795
Non-trainable params: 0
_________________________________________________________________
샘플링을 시작합니다!
 ss           printf(" .. MAX_LOCK_USASHIC_PRINT_TASK_TIMEKEEntry))
		if (diag) {
		reset_free_percpu(skb);
		again_ops->proc_cap_buffer = bytesn;
	}

	/*
	 * Determine if CONFIG_SCHEDSTATS */

	/* Keep start */
	dr->flags &= ~SPONT_PAREN; i++) {
		kdb_printf("kdb enabled" by this code trigger that may have been actually. If all
 * threads can possibly get_kprojid to